In [7]:
%pip install av
! wget https://raw.githubusercontent.com/pytorch/vision/6de158c473b83cf43344a0651d7c01128c7850e6/references/video_classification/transforms.py
# Download HMDB51 data and splits from serre lab website
! wget http://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/hmdb51_org.rar
! wget http://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/test_train_splits.rar

Note: you may need to restart the kernel to use updated packages.
--2023-08-29 14:05:40--  https://raw.githubusercontent.com/pytorch/vision/6de158c473b83cf43344a0651d7c01128c7850e6/references/video_classification/transforms.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3102 (3.0K) [text/plain]
Saving to: ‘transforms.py.1’

transforms.py.1     100%[===================>]   3.03K  --.-KB/s    in 0s      

2023-08-29 14:05:40 (38.7 MB/s) - ‘transforms.py.1’ saved [3102/3102]

--2023-08-29 14:05:40--  http://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/hmdb51_org.rar
Resolving serre-lab.clps.brown.edu (serre-lab.clps.brown.edu)... 128.148.254.114
Connecting to serre-lab.clps.brown.edu (serre-lab.clps.brown.edu)|128.148.254.114|:80... connected.
HTTP r

In [3]:
%pip uninstall -y torch

Found existing installation: torch 2.0.1
Uninstalling torch-2.0.1:
  Successfully uninstalled torch-2.0.1
Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install torch==2.0.1+cu118 --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
     |████████████████████████████████| 2267.3 MB 1.5 kB/s s eta 0:00:01�███▌                      | 675.1 MB 60.9 MB/s eta 0:00:27     |███████████▎                    | 801.4 MB 129.6 MB/s eta 0:00:12
Note: you may need to restart the kernel to use updated packages.


In [6]:
%pip install git+https://github.com/Atze00/MoViNet-pytorch.git
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

  Cloning https://github.com/Atze00/MoViNet-pytorch.git to /tmp/pip-req-build-keokn1yi
  Running command git clone -q https://github.com/Atze00/MoViNet-pytorch.git /tmp/pip-req-build-keokn1yi
  Created wheel for MoViNet-pytorch: filename=MoViNet_pytorch-0.2.2-py3-none-any.whl size=11960 sha256=5299eeb37e6124e919b5f98492ff671cdd7f2aa2c3a676da4f6847b7d1831673
  Stored in directory: /tmp/pip-ephem-wheel-cache-d025k6kg/wheels/ee/bf/35/16fde806d3c2077ab3ae9c186525fab12b5f00240a6c1b351c
Successfully built MoViNet-pytorch
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://download.pytorch.org/whl/cu118
Note: you may need to restart the kernel to use updated packages.


In [8]:
# Extract and organize video data..
! mkdir -p video_data test_train_splits
! unrar e test_train_splits.rar test_train_splits
! rm test_train_splits.rar
! unrar e hmdb51_org.rar
! rm hmdb51_org.rar
! mv *.rar video_data
import os
for files in os.listdir('video_data'):
    foldername = files.split('.')[0]
    os.system("mkdir -p video_data/" + foldername)
    os.system("unrar e video_data/"+ files + " video_data/"+foldername)

! rm video_data/*.rar



UNRAR 5.61 beta 1 freeware      Copyright (c) 1993-2018 Alexander Roshal


Extracting from test_train_splits.rar

Extracting  test_train_splits/brush_hair_test_split1.txt                 0  OK 
Extracting  test_train_splits/brush_hair_test_split2.txt                 1  OK 
Extracting  test_train_splits/brush_hair_test_split3.txt                 1  OK 
Extracting  test_train_splits/cartwheel_test_split1.txt                  2  OK 
Extracting  test_train_splits/cartwheel_test_split2.txt                  2  OK 
Extracting  test_train_splits/cartwheel_test_split3.txt                  3  OK 
Extracting  test_train_splits/catch_test_split1.txt                      4  OK 
Extracting  test_train_splits/catch_test_split2.txt                      4  OK 
Extracting  test_train_splits/catch_test_split3.txt                      5  OK 
Extracting  test_train_splits/chew_test_split1.txt                       5  OK 
Extracting  test_train_splits/chew_test_split2.txt                       6  OK 
Extra

In [1]:
import time
import torchvision
import torch.nn.functional as F
import torchvision.transforms as transforms
import torch.optim as optim
from torch.utils.data import random_split, DataLoader
import torch
import transforms as T
from movinets import MoViNet
from movinets.config import _C

torch.manual_seed(97)
num_frames = 16 # 16
clip_steps = 2
Bs_Train = 16
Bs_Test = 16

transform = transforms.Compose([

                                 T.ToFloatTensorInZeroOne(),
                                 T.Resize((200, 200)),
                                 T.RandomHorizontalFlip(),
                                 #T.Normalize(mean=[0.43216, 0.394666, 0.37645], std=[0.22803, 0.22145, 0.216989]),
                                 T.RandomCrop((172, 172))])
transform_test = transforms.Compose([
                                 T.ToFloatTensorInZeroOne(),
                                 T.Resize((200, 200)),
                                 #T.Normalize(mean=[0.43216, 0.394666, 0.37645], std=[0.22803, 0.22145, 0.216989]),
                                 T.CenterCrop((172, 172))])

hmdb51_train = torchvision.datasets.HMDB51('video_data/', 'test_train_splits/', num_frames,frame_rate=5,
                                                step_between_clips = clip_steps, fold=1, train=True,
                                                transform=transform, num_workers=2)


hmdb51_test = torchvision.datasets.HMDB51('video_data/', 'test_train_splits/', num_frames,frame_rate=5,
                                                step_between_clips = clip_steps, fold=1, train=False,
                                                transform=transform_test, num_workers=2)
train_loader = DataLoader(hmdb51_train, batch_size=Bs_Train, shuffle=True)
test_loader  = DataLoader(hmdb51_test, batch_size=Bs_Test, shuffle=False)

100%|██████████| 423/423 [00:37<00:00, 11.24it/s]
/home/wldnjs/.venv/lib/python3.8/site-packages/torchvision/datasets/video_utils.py:219: UserWarning: There aren't enough frames in the current video to get a clip for the given clip length and frames between clips. The video (and potentially others) will be skipped.
  warnings.warn(
100%|██████████| 423/423 [00:39<00:00, 10.77it/s]


In [4]:
def train_iter(model, optimz, data_load, loss_val):
    samples = len(data_load.dataset)
    model.train()
    model.cuda()
    model.clean_activation_buffers()
    optimz.zero_grad()
    for i, (data,_ , target) in enumerate(data_load):
        out = F.log_softmax(model(data.cuda()), dim=1)
        loss = F.nll_loss(out, target.cuda())
        loss.backward()
        optimz.step()
        optimz.zero_grad()
        model.clean_activation_buffers()
        if i % 50 == 0:
            print('[' +  '{:5}'.format(i * len(data)) + '/' + '{:5}'.format(samples) +
                  ' (' + '{:3.0f}'.format(100 * i / len(data_load)) + '%)]  Loss: ' +
                  '{:6.4f}'.format(loss.item()))
            loss_val.append(loss.item())

def evaluate(model, data_load, loss_val):
    model.eval()

    samples = len(data_load.dataset)
    csamp = 0
    tloss = 0
    model.clean_activation_buffers()
    with torch.no_grad():
        for data, _, target in data_load:
            output = F.log_softmax(model(data.cuda()), dim=1)
            loss = F.nll_loss(output, target.cuda(), reduction='sum')
            _, pred = torch.max(output, dim=1)

            tloss += loss.item()
            csamp += pred.eq(target.cuda()).sum()
            model.clean_activation_buffers()
    aloss = tloss / samples
    loss_val.append(aloss)
    print('\nAverage test loss: ' + '{:.4f}'.format(aloss) +
          '  Accuracy:' + '{:5}'.format(csamp) + '/' +
          '{:5}'.format(samples) + ' (' +
          '{:4.2f}'.format(100.0 * csamp / samples) + '%)\n')

def train_iter_stream(model, optimz, data_load, loss_val, n_clips = 2, n_clip_frames=8):
    """
    In causal mode with stream buffer a single video is fed to the network
    using subclips of lenght n_clip_frames.
    n_clips*n_clip_frames should be equal to the total number of frames presents
    in the video.

    n_clips : number of clips that are used
    n_clip_frames : number of frame contained in each clip
    """
    #clean the buffer of activations
    samples = len(data_load.dataset)
    model.cuda()
    model.train()
    model.clean_activation_buffers()
    optimz.zero_grad()

    for i, (data,_, target) in enumerate(data_load):
        data = data.cuda()
        target = target.cuda()
        l_batch = 0
        #backward pass for each clip
        for j in range(n_clips):
          output = F.log_softmax(model(data[:,:,(n_clip_frames)*(j):(n_clip_frames)*(j+1)]), dim=1)
          loss = F.nll_loss(output, target)
          _, pred = torch.max(output, dim=1)
          loss = F.nll_loss(output, target)/n_clips
          loss.backward()
        l_batch += loss.item()*n_clips
        optimz.step()
        optimz.zero_grad()

        #clean the buffer of activations
        model.clean_activation_buffers()
        if i % 50 == 0:
            print('[' +  '{:5}'.format(i * len(data)) + '/' + '{:5}'.format(samples) +
                  ' (' + '{:3.0f}'.format(100 * i / len(data_load)) + '%)]  Loss: ' +
                  '{:6.4f}'.format(l_batch))
            loss_val.append(l_batch)

def evaluate_stream(model, data_load, loss_val, n_clips = 2, n_clip_frames=8):
    model.eval()
    model.cuda()
    samples = len(data_load.dataset)
    csamp = 0
    tloss = 0
    with torch.no_grad():
        for data, _, target in data_load:
            data = data.cuda()
            target = target.cuda()
            model.clean_activation_buffers()
            for j in range(n_clips):
              output = F.log_softmax(model(data[:,:,(n_clip_frames)*(j):(n_clip_frames)*(j+1)]), dim=1)
              loss = F.nll_loss(output, target)
            _, pred = torch.max(output, dim=1)
            tloss += loss.item()
            csamp += pred.eq(target).sum()

    aloss = tloss /  len(data_load)
    loss_val.append(aloss)
    print('\nAverage test loss: ' + '{:.4f}'.format(aloss) +
          '  Accuracy:' + '{:5}'.format(csamp) + '/' +
          '{:5}'.format(samples) + ' (' +
          '{:4.2f}'.format(100.0 * csamp / samples) + '%)\n')

In [10]:
model = MoViNet(_C.MODEL.MoViNetA0, causal = True, pretrained = True )

trloss_val, tsloss_val = [], []
model.classifier[3] = torch.nn.Conv3d(2048, 51, (1,1,1))
optimz = optim.RMSprop(model.parameters(), lr=0.0001)
scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimz, 5)

In [11]:
N_EPOCHS = 10
train_steps = len(train_loader.dataset)

start_time = time.time()
for epoch in range(1, N_EPOCHS + 1):
    print('Epoch:', epoch, '/', N_EPOCHS)
    train_iter_stream(model, optimz, train_loader, trloss_val)
    evaluate_stream(model, test_loader, tsloss_val)
    scheduler.step()

print('Execution time:', '{:5.2f}'.format(time.time() - start_time), 'seconds')

Epoch: 1 / 10
[    0/ 7519 (  0%)]  Loss: 3.9705
[  800/ 7519 ( 11%)]  Loss: 1.8427
[ 1600/ 7519 ( 21%)]  Loss: 0.6428
[ 2400/ 7519 ( 32%)]  Loss: 1.2057
[ 3200/ 7519 ( 43%)]  Loss: 0.4144
[ 4000/ 7519 ( 53%)]  Loss: 0.2919
[ 4800/ 7519 ( 64%)]  Loss: 0.3894
[ 5600/ 7519 ( 74%)]  Loss: 0.8799
[ 6400/ 7519 ( 85%)]  Loss: 0.3226
[ 7200/ 7519 ( 96%)]  Loss: 0.3633

Average test loss: 0.9415  Accuracy: 2264/ 2944 (76.90%)

Epoch: 2 / 10
[    0/ 7519 (  0%)]  Loss: 0.0750
[  800/ 7519 ( 11%)]  Loss: 0.3298
[ 1600/ 7519 ( 21%)]  Loss: 0.4962
[ 2400/ 7519 ( 32%)]  Loss: 0.6742
[ 3200/ 7519 ( 43%)]  Loss: 0.2365
[ 4000/ 7519 ( 53%)]  Loss: 0.1184
[ 4800/ 7519 ( 64%)]  Loss: 0.2489
[ 5600/ 7519 ( 74%)]  Loss: 0.2545
[ 6400/ 7519 ( 85%)]  Loss: 0.1224
[ 7200/ 7519 ( 96%)]  Loss: 0.0153

Average test loss: 0.9149  Accuracy: 2182/ 2944 (74.12%)

Epoch: 3 / 10
[    0/ 7519 (  0%)]  Loss: 0.8242
[  800/ 7519 ( 11%)]  Loss: 0.0550
[ 1600/ 7519 ( 21%)]  Loss: 0.4125
[ 2400/ 7519 ( 32%)]  Loss: 0.0737


In [ ]:
N_EPOCHS = 1

model = MoViNet(_C.MODEL.MoViNetA0, causal = False, pretrained = True )
start_time = time.time()

trloss_val, tsloss_val = [], []
model.classifier[3] = torch.nn.Conv3d(2048, 51, (1,1,1))
optimz = optim.Adam(model.parameters(), lr=0.00005)
for epoch in range(1, N_EPOCHS + 1):
    print('Epoch:', epoch)
    train_iter(model, optimz, train_loader, trloss_val)
    evaluate(model, test_loader, tsloss_val)

print('Execution time:', '{:5.2f}'.format(time.time() - start_time), 'seconds')